In [1]:
from TALib import TALib

In [2]:
ta_lib = TALib()

Map:   0%|          | 0/15159 [00:00<?, ? examples/s]

Map:   0%|          | 0/3790 [00:00<?, ? examples/s]

## Load model

In [3]:
model = ta_lib.load_model("./output/pruning_success_v2_max_128")

Some weights of the model checkpoint at ./output/pruning_success_v2_max_128 were not used when initializing T5ForConditionalGeneration: ['decoder.block.0.layer.0.SelfAttention.k.weight_mask', 'decoder.block.0.layer.0.SelfAttention.k.weight_orig', 'decoder.block.0.layer.0.SelfAttention.o.weight_mask', 'decoder.block.0.layer.0.SelfAttention.o.weight_orig', 'decoder.block.0.layer.0.SelfAttention.q.weight_mask', 'decoder.block.0.layer.0.SelfAttention.q.weight_orig', 'decoder.block.0.layer.0.SelfAttention.v.weight_mask', 'decoder.block.0.layer.0.SelfAttention.v.weight_orig', 'decoder.block.0.layer.1.EncDecAttention.k.weight_mask', 'decoder.block.0.layer.1.EncDecAttention.k.weight_orig', 'decoder.block.0.layer.1.EncDecAttention.o.weight_mask', 'decoder.block.0.layer.1.EncDecAttention.o.weight_orig', 'decoder.block.0.layer.1.EncDecAttention.q.weight_mask', 'decoder.block.0.layer.1.EncDecAttention.q.weight_orig', 'decoder.block.0.layer.1.EncDecAttention.v.weight_mask', 'decoder.block.0.layer.1

In [4]:
model

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

## Display model compress ratio

In [5]:
TALib.show_param_ratio(model)

0.26261115074157715

## Get Trainer

In [6]:
trainer = ta_lib.get_trainer(
    model=model , 
    num_train_epochs=10 , 
    batch_size=5 , 
    output_dir="output/pruning_success_v2_finetune" , 
    generation_max_length=128
)

/root/code/python/NYCU-Data-Science-2024/.venv/lib/python3.11/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


## Evaluate

In [7]:
trainer.evaluate(ta_lib.tokenized_billsum_test , max_length=128)

KeyboardInterrupt: 

## Predict

In [8]:
results = trainer.predict(
    ta_lib.tokenized_billsum_test , 
    max_length=128 , 
    num_beams=10,
    early_stopping=True,
    do_sample=True,
    top_k=50,
    top_p=0.95,
    no_repeat_ngram_size=2,
)

In [9]:
import numpy as np 
results = np.where(results[0] != -100, results[0], ta_lib.tokenizer.pad_token_id)

In [10]:
decoded_prediction = ta_lib.tokenizer.batch_decode(results, skip_special_tokens=True)

## To Ta Kaggle Summit format

In [11]:
TALib.dump_to_kaggle_format(decoded_prediction , 'pruned_model_0.26_max_128_beam_10_gen_super.csv')

,ID,Predict
0,0,Amends the Water Resources Development Act of ...
1,1,Federal Forage Fee Act of 1993 - Amends the Fe...
2,2,Merchant Marine of World War II Congressional ...
3,3,Small Business Tax Modernization Act of 2004 -...
4,4,Fair Access to Investment Research Act of 2016...
...,...,...
3264,3264,Public Servant Priority Placement Act of 1995 ...
3265,3265,Sportsmanship in Hunting Act of 2008 - Amends ...
3266,3266,Helping College Student Cross the Finish Line ...
3267,3267,Texas National Forests Improvement Act of 2000...


## Run TA Score

In [12]:
TALib.run_score(decoded_prediction,ta_lib.tokenized_billsum_test)

0.3283940623183895